In [ ]:

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
import os

deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

kernel = sk.Kernel()

kernel.add_chat_service(                      # We are adding a text service
    "Azure_curie",                            # The alias we can use in prompt templates' config.json
    AzureChatCompletion(
        "my-finetuned-Curie",                 # Azure OpenAI *Deployment name*
        "https://contoso.openai.azure.com/",  # Azure OpenAI *Endpoint*
        "...your Azure OpenAI Key..."         # Azure OpenAI *Key*
    )
)

### Running Semantic Functions Inline
Try Using ChatCompletion for Semantic Skills which can use Newer models (2023–)	gpt-4, gpt-4 turbo, gpt-3.5-turbo\
Both AzureTextCompletion and OpenAITextCompletion currently only use Legacy models (2020–2022)	text-davinci-003, text-davinci-002, davinci, curie, babbage, ada

In [34]:
import os
import semantic_kernel as sk
# from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

# OpenAIChatCompletion is more cheaper than AzureChatCompletion
useAzureOpenAI = False
api_key = os.getenv("OPENAI_API_KEY")
org_id = os.getenv("OPENAI_ORG_ID")
model = "text-davinci-003"

# Configure AI service used by the kernel
# if useAzureOpenAI:
#     # deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
#     kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
# else:
#     # api_key, org_id = sk.openai_settings_from_dot_env()
#     kernel.add_text_completion_service("dv", OpenAITextCompletion(model, api_key, org_id))

# Configure AI service used by the kernel
if useAzureOpenAI:
    # deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion(deployment, endpoint, api_key),
    )
else:
    # api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id)
    )

In [ ]:
prompt = """{{$input}}
Summarize the content above.
"""

summarize = kernel.create_semantic_function(prompt, max_tokens=2000, temperature=0.2, top_p=0.5)

In [ ]:
input_text = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
"""

In [ ]:
# If needed, async is available too: summary = await summarize.invoke_async(input_text)
summary = summarize(input_text)

print(summary)

### Creating a basic chat experience with context variables
environments initialized is on the top paragraph

In [ ]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [ ]:
# Register your semantic function
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

# Initialize your context
context = kernel.create_new_context()
context["history"] = ""

In [ ]:
# Chat with the Bot
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = None
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

continue_flag = False

In [68]:
import json

if continue_flag:
    def show_json(obj):
        display(json.loads(obj.json())) 

    # Pretty printing helper
    def pretty_print(messages):
        print("# Messages")
        for m in messages:
            print(f"{m.role}: {m.content[0].text.value}")
        print()

    show_json(bot_answer)
    # print(bot_answer)

In [ ]:
# Update the history with the output
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])

In [ ]:
# Keep Chatting!
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [ ]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

In [ ]:
await chat("that sounds interesting, what is it about?")

In [ ]:
await chat("if I read that book, what exactly will I learn about Greek history?")

In [ ]:
await chat("could you list some more books I could read about this topic?")

In [ ]:
print(context["history"])

### Introduction to the Planner
planner works best with more powerful models like gpt4 but sometimes you might get working plans with cheaper models like gpt-35-turbo, since planner will output json or xml format and gpt4 can handle that perfectly\
examples are available in 08-using-the-planner.ipynb

### Building Semantic Memory with Embeddings
If prompt would grow so large that you would run into a the model's token limit. What we need is a way to persist state and build both short-term and long-term memory to empower even more intelligent applications.

In [27]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding, AzureChatCompletion, AzureTextEmbedding

In order to use memory, we need to instantiate the Kernel with a Memory Storage and an Embedding service. In this example, we make use of the VolatileMemoryStore "which can be thought of as a temporary in-memory storage (not to be confused with Semantic Memory). This memory is not written to disk and is only available during the app session.

When developing your app you will have the option to plug in persistent storage like Azure Cosmos Db, PostgreSQL, SQLite, etc. Semantic Memory allows also to index external data sources, without duplicating all the information, more on that later.

In [29]:
kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    # deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
    # next line assumes embeddings deployment name is "text-embedding-ada-002", adjust this if  appropriate 
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", endpoint, api_key))
else:
    # api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x190d18330d0>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x190d1833550>}

At its core, Semantic Memory is a set of data structures that allow you to store the meaning of text that come from different data sources, and optionally to store the source text too. These texts can be from the web, e-mail providers, chats, a database, or from your local directory, and are hooked up to the Semantic Kernel through data source connectors.

In [30]:
# Let's create some initial memories "About Me". 
# We can add memories to our VolatileMemoryStore by using SaveInformationAsync
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(
        "aboutMe", id="info1", text="My name is Andrea"
    )
    # await kernel.memory.save_information_async(
    #     "aboutMe", id="info2", text="I currently work as a tour guide"
    # )
    # await kernel.memory.save_information_async(
    #     "aboutMe", id="info3", text="I've been living in Seattle since 2005"
    # )
    # await kernel.memory.save_information_async(
    #     "aboutMe", id="info4", text="I visited France and Italy five times since 2015"
    # )
    # await kernel.memory.save_information_async(
    #     "aboutMe", id="info5", text="My family is from New York"
    # )

In [32]:
# Let's try searching the memory:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search_async("aboutMe", question)
        print(f"Answer: {result[0].text}\n")

More steps are found in https://github.com/microsoft/semantic-kernel/blob/main/python/notebooks/06-memory-and-embeddings.ipynb

### Running Native Functions
This can be useful in a few scenarios:

* Writing logic around how to run a prompt that changes the prompt's outcome.
* Using external data sources to gather data to concatenate into your prompt.
* Validating user input data prior to sending it to the LLM prompt.

In [44]:
import random
from semantic_kernel.skill_definition import sk_function

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher"
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            print("inside function") # debug print
            return str(random.randint(3, int(input))) 
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

In [49]:
# Next, let's create a semantic function that accepts a number as {{$input}} 
# and generates that number of paragraphs about two Corgis on an adventure. 
# $input is a default variable semantic functions can use.
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long
"""

corgi_story = kernel.create_semantic_function(prompt_template=sk_prompt,
                                              function_name="CorgiStory",
                                              skill_name="CorgiSkill",
                                              description="Write a short story about two Corgis on an adventure",
                                              max_tokens=500,
                                              temperature=0.5,
                                              top_p=0.5)

generate_number_skill = kernel.import_skill(GenerateNumberSkill())

In [50]:
# Run the number generator
generate_number_three_or_higher = generate_number_skill["GenerateNumberThreeOrHigher"]
number_result = generate_number_three_or_higher(6)
print(number_result)
# generate_number_three_or_higher.name

inside function
5


In [51]:
story = await corgi_story.invoke_async(input=number_result.result)
print("Generating a corgi story exactly {} paragraphs long: ".format(number_result.result))
print("=====================================================")
print(story)

Generating a corgi story exactly 5 paragraphs long: 
Once upon a time in a cozy little town, there lived two adorable Corgis named Charlie and Daisy. They were the best of friends and loved going on adventures together. One sunny morning, they decided to explore the nearby forest, filled with tall trees and colorful flowers.

As they trotted along the winding path, Charlie and Daisy noticed a lost butterfly fluttering around in confusion. Worried, they approached the butterfly and asked if it needed help. The butterfly explained that it had lost its way home and was feeling scared.

Without hesitation, Charlie and Daisy offered their assistance. They led the butterfly through the forest, using their keen sense of smell and sharp eyes to find familiar landmarks. Along the way, they encountered other animals who joined their mission, creating a little parade of kindness.

Finally, they reached a beautiful meadow where the butterfly's family eagerly awaited its return. The butterfly thank

#### Context Variables
That works! But let's expand on our example to make it more generic.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of <strong>the semantic_kernel.ContextVariables class</strong> to do hold these variables.

Let's start with the native function. Notice that we're also adding @sk_function_context_parameter decorators to the function here to provide context about what variables need to be provided to the function, and any defaults for those inputs. Using the @sk_function_context_parameter decorator provides the name, description and default values for a function's inputs to the planner.

In [52]:
import random
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext

class GenerateNumberSkill:
    """
    Description: Generate a number between a min and a max.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber"
    )
    @sk_function_context_parameter(name="min", description="Minimum number of paragraphs.")
    @sk_function_context_parameter(name="max", description="Maximum number of paragraphs.", default_value=10)
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(int(context["min"]), int(context["max"]))) 
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

In [53]:
generate_number_skill = kernel.import_skill(GenerateNumberSkill())
generate_number = generate_number_skill["GenerateNumber"]

In [54]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

corgi_story = kernel.create_semantic_function(prompt_template=sk_prompt,
                                              function_name="CorgiStory",
                                              skill_name="CorgiSkill",
                                              description="Write a short story about two Corgis on an adventure",
                                              max_tokens=500,
                                              temperature=0.5,
                                              top_p=0.5)

In [56]:
context_variables = sk.ContextVariables(variables={
    "min": 1,
    "max": 5,
    "language": "Chinese",
})

context_variables['paragraph_count'] = generate_number.invoke(variables=context_variables).result

# Pass the output to the semantic story function
story = await corgi_story.invoke_async(variables=context_variables)

print("Generating a corgi story exactly {} paragraphs long in {} language: ".format(context_variables["paragraph_count"],
                                                                                    context_variables["language"]))
print("=====================================================")
print(story)

Generating a corgi story exactly 4 paragraphs long in Chinese language: 
从小到大，小短腿和小长腿一直是最好的朋友。他们都是可爱的柯基犬，每天都在花园里一起玩耍。然而，他们总是梦想着有一天能够去探险，看看世界的其他地方。

有一天，他们决定实现自己的梦想。小短腿和小长腿穿过了大草原，跳过了小溪，来到了一个陌生的森林。他们充满好奇地探索着，发现了一只迷路的小松鼠。小短腿和小长腿决定帮助它找到回家的路。

他们穿过了茂密的树林，跳过了高高的石头，终于找到了小松鼠的家。小松鼠非常感激，它邀请小短腿和小长腿在森林里玩耍。他们一起玩耍，跳跃，享受着快乐的时光。

当天晚上，小短腿和小长腿回到了自己的家。他们感到非常满足和幸福，因为他们不仅实现了自己的梦想，还帮助了别人。他们明白到，友谊和善良是最重要的事情，无论身处何地。

从那以后，小短腿和小长腿成为了森林和花园之间的桥梁。他们带着快乐和友谊的故事，与其他动物分享。他


#### Calling Native Functions within a Semantic Function
One neat thing about the Semantic Kernel is that you can also call native functions from within Semantic Functions!

We will make our CorgiStory semantic function call a native function GenerateNames which will return names for our Corgi characters.

We do this using the syntax {{skill_name.function_name}}. You can read more about our prompte templating syntax here.

In [60]:
import os
import semantic_kernel as sk
# from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

# OpenAIChatCompletion is more cheaper than AzureChatCompletion
useAzureOpenAI = False
api_key = os.getenv("OPENAI_API_KEY")
org_id = os.getenv("OPENAI_ORG_ID")
model = "text-davinci-003"

# Configure AI service used by the kernel
# if useAzureOpenAI:
#     # deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
#     kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
# else:
#     # api_key, org_id = sk.openai_settings_from_dot_env()
#     kernel.add_text_completion_service("dv", OpenAITextCompletion(model, api_key, org_id))

# Configure AI service used by the kernel
if useAzureOpenAI:
    # deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion(deployment, endpoint, api_key),
    )
else:
    # api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id)
    )

In [61]:
import random
from semantic_kernel.skill_definition import sk_function


class GenerateNamesSkill:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @sk_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @sk_function(
        description="Generate character names",
        name="generate_names"
    )
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {
            "Hoagie",
            "Hamilton",
            "Bacon",
            "Pizza",
            "Boots",
            "Shorts",
            "Tuna"
        }
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [62]:
generate_names_skill = kernel.import_skill(GenerateNamesSkill(), skill_name="GenerateNames")
generate_names = generate_names_skill["generate_names"]

In [63]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The two names of the corgis are {{GenerateNames.generate_names}}
"""

In [64]:
corgi_story = kernel.create_semantic_function(prompt_template=sk_prompt,
                                              function_name="CorgiStory",
                                              skill_name="CorgiSkill",
                                              description="Write a short story about two Corgis on an adventure",
                                              max_tokens=500,
                                              temperature=0.5,
                                              top_p=0.5)

In [65]:
context_variables = sk.ContextVariables(variables={
    "min": 1,
    "max": 5,
    "language": "Chinese",
})
context_variables['paragraph_count'] = generate_number.invoke(variables=context_variables).result

# Pass the output to the semantic story function
story = await corgi_story.invoke_async(variables=context_variables)

print("Generating a corgi story exactly {} paragraphs long in {} language: ".format(context_variables["paragraph_count"],
                                                                                    context_variables["language"]))
print("=====================================================")
print(story)

Generating a corgi story exactly 2 paragraphs long in Chinese language: 
Bacon和Tuna是两只可爱的柯基犬，它们是最好的朋友。一天，它们决定展开一次冒险之旅。它们在阳光明媚的早晨出发了。

它们穿过了郁郁葱葱的森林，跳过了清澈的小溪，一直走到了一个美丽的花园。在那里，它们看到了一只小鸟被困在了网中。Bacon和Tuna立刻决定帮助它。它们小心翼翼地咬断了网子，小鸟终于自由了。小鸟欢快地唱起歌来，感激地飞走了。

Bacon和Tuna感到非常快乐和满足。它们明白了一个重要的道理：帮助别人会带来快乐。它们继续它们的冒险之旅，希望能够帮助更多的朋友。从那天起，Bacon和Tuna成为了这个小镇上最受欢迎的狗狗，它们的友谊和善良的行为一直鼓舞着大家。
